In [1]:
from math import floor

def max_pool(height, kernel_size, stride=None, padding=0, dilation=1):
    if stride is None:
        stride = kernel_size
    return floor((height + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)

def conv(height, kernel_size, stride=1, padding=0, dilation=1):
    return floor((height + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)

In [2]:
sizes = [conv(400, 3)]
for i in range(0, 8, 2):
    sizes.append(max_pool(sizes[i], 2, stride=2))
    sizes.append(conv(sizes[i+1], 3))
sizes.append(max_pool(sizes[8], 2, stride=2))
[s ** 2 for s in sizes]

[158404, 39601, 38809, 9604, 9216, 2304, 2116, 529, 441, 100]

In [3]:
sizes

[398, 199, 197, 98, 96, 48, 46, 23, 21, 10]

In [4]:
import torch

In [11]:
torch.randn(20, 100, 35, 45).size()

torch.Size([20, 100, 35, 45])